In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.insert(0, "/notebooks/")
from fastai.imports import *

In [2]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
import pandas

In [4]:
PATH = "/notebooks/data/cars/"
IMG_PATH = "/notebooks/data/cars/cars_train/"
sz=224

In [5]:
label_csv = f'/notebooks/notebook/projects/cars/train.csv'

In [6]:
n = len(list(open(label_csv)))-1
val_idxs = get_cv_idxs(n)

In [7]:
f_model = resnet34

In [8]:
def get_data(sz):
    tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_basic, max_zoom=1.05)
    return ImageClassifierData.from_csv(PATH, 'cars_train', label_csv, tfms=tfms, val_idxs=val_idxs)

In [9]:
torch.cuda.is_available()

True

In [10]:
torch.backends.cudnn.enabled

True

In [11]:
sz=448

In [12]:
data = get_data(sz)

In [13]:
learn = ConvLearner.pretrained(f_model, data)

In [14]:
lr = 1e-1

In [15]:
learn.precompute=False

In [16]:
learn.unfreeze()

In [17]:
lrs = np.array([lr/1e10,lr/1e5,lr])

In [18]:
learn.load(f'first_{sz}')

In [19]:
learn.set_data(get_data(sz))

In [20]:
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   accuracy   
    0      1.269497   1.919067   0.512285  
    1      1.375771   1.93021    0.507985  
    2      1.177444   1.902276   0.520885  
    3      1.34001    1.956414   0.506143  
    4      1.270752   1.920343   0.516585  
    5      1.105461   1.86182    0.527027  
    6      0.985415   1.862582   0.523342  



[array([1.86258]), 0.5233415231950745]

In [21]:
learn.save(f'second_{sz}')

In [22]:
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)

In [23]:
accuracy_np(probs, y)

0.5743243243243243

In [25]:
len(data.classes)

196